In [544]:
#Imports
import spotipy, json, webbrowser, urllib.request
import pandas as pd
import spotipy.util as util
import random
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate




In [545]:
#Cocktail Cell 1
kseniia_lucky_number = 5
omotara_lucky_number = 3

#Manual input, comment the below lines out when ready
user_input = [["Dynamic", "Spicy"],  ["Beef"], ["Thai", "Indian", "Korean"], [28, 11, 23]]
date_vibe = user_input[0]
dietary_req = user_input[1]
pref_cuisines = user_input[2]
date_day = user_input[3][0]
date_month = user_input[3][1]
date_year = user_input[3][2]

#Dynamic input, uncomment the below lines later
# true_date_vibe = input("Tell me about your date... \n What vibe do you want? \n Describe it in a list (e.g. ['Dynamic', 'Spicy'])")
# true_dietary_req = input("And what are your dietary requirements? \n Give them in a list (e.g. ['Beef'])\n")
# true_pref_cuisines = input("What are your preferred cuisines? \n Write them as a list (e.g. ['Italian', 'Korean'])\n")
# true_date = input("And finally when is the date? \n Format it like this [dd, mm, yy] (e.g. [28, 11, 23])\n")
# date_day = true_date[0]
# date_month = true_date[1]
# date_year = true_date[2]

In [546]:
#Cocktail Cell 2
letters = []

for word in date_vibe:
    for char in word:
        letters.append(char.lower())

print(letters)

['d', 'y', 'n', 'a', 'm', 'i', 'c', 's', 'p', 'i', 'c', 'y']


In [547]:
#Cocktail Cell 3
cocktail_suggestions = []
for letter in letters:
    number = kseniia_lucky_number
    query_str = f"http://www.thecocktaildb.com/api/json/v1/1/search.php?f={letter}"
    request = urllib.request.Request(query_str)
    response = urllib.request.urlopen(request)
    all_cocktails = json.loads(response.read())
    if len(all_cocktails['drinks']) < (number-1):
        number = len(all_cocktails['drinks'])
    cocktail = all_cocktails['drinks']
    if cocktail[0]['strDrink'] not in cocktail_suggestions:
        cocktail_name = cocktail[0]['strDrink']
        cocktail_instructions = cocktail[0]['strInstructions']
        cocktail_ingred = cocktail[0]['strIngredient1'].lower()
        cocktail_img = cocktail[0]['strDrinkThumb']
        num = 2
        while cocktail[0][f'strIngredient{num}'] != None:
            cocktail_ingred = cocktail_ingred + ", " + cocktail[0][f'strIngredient{num}'].lower()
            num += 1
        cocktail_suggestions.append([cocktail_name, cocktail_instructions, cocktail_ingred, cocktail_img])
print(cocktail_suggestions)

[['Derby', 'Pour all ingredients into a mixing glass with ice. Stir. Strain into a cocktail glass. Garnish with a sprig of fresh mint in the drink.', 'gin, peach bitters, mint', 'https://www.thecocktaildb.com/images/media/drink/52weey1606772672.jpg'], ['Yellow Bird', 'Shake and strain into a chilled cocktail glass', 'white rum, galliano, triple sec, lime juice', 'https://www.thecocktaildb.com/images/media/drink/2t9r6w1504374811.jpg'], ['Negroni', 'Stir into glass over ice, garnish and serve.', 'gin, campari, sweet vermouth', 'https://www.thecocktaildb.com/images/media/drink/qgdu971561574065.jpg'], ['A1', 'Pour all ingredients into a cocktail shaker, mix and serve over ice into a chilled glass.', 'gin, grand marnier, lemon juice, grenadine', 'https://www.thecocktaildb.com/images/media/drink/2x8thr1504816928.jpg'], ['Melya', 'Brew espresso. In a coffee mug, place 1 teaspoon of unsweetened powdered cocoa, then cover a teaspoon with honey and drizzle it into the cup. Stir while the coffee 

In [548]:
#Cocktail Cell 4
number = omotara_lucky_number
random.shuffle(cocktail_suggestions)
if len(cocktail_suggestions) < number:
    number = len(cocktail_suggestions)
your_cocktail = cocktail_suggestions[number][0]
your_recipe = cocktail_suggestions[number][1]
your_ingredients = cocktail_suggestions[number][2]
your_cocktail_img = cocktail_suggestions[number][3]

a_or_an = "a"
vowels = ["a", "e", "i", "o", "u"]
if your_cocktail[0].lower() in vowels:
    a_or_an = "an"
    
print(f"Your cocktail for the evening is {a_or_an}\033[1m {your_cocktail}\n\033[0m Here's the recipe \n Ingredients: {your_ingredients}\n Steps: {your_recipe}")

Your cocktail for the evening is a Casino
 Here's the recipe 
 Ingredients: gin, maraschino liqueur, lemon juice, orange bitters, cherry
 Steps: Pour all ingredients into shaker with ice cubes. Shake well. Strain into chilled cocktail glass. Garnish with a lemon twist and a maraschino cherry. Serve without a straw.


In [549]:
#Meal

In [550]:
#Playlist

In [560]:
#PDF
top_line_pos = 750
line_width = 20
date_details = "28 Nov"
date_canvas = canvas.Canvas(f"date_results/{date_details.replace(" ", "_")}_date_night.pdf", pagesize=A4)
date_canvas.drawString(100, top_line_pos, "MENU")
test = "aygyuwgefkjhdkjhkskjdewkjhd"

def line_y_position(line_idx):
    return top_line_pos-(line_idx*line_width)

#Cocktail Section
cocktail_img_item = ImageReader(your_cocktail_img)
position = 1
date_canvas.drawString(100, line_y_position(position), f"Your cocktail for the evening is {a_or_an} {your_cocktail}")
position += 1
date_canvas.drawImage(cocktail_img_item, 100, line_y_position(position + 10), mask='auto', width=100, height=100)
position += 1
date_canvas.drawString(100, line_y_position(position), f"Ingredients: {your_ingredients}")
position += 1
date_canvas.drawString(100, line_y_position(position), f"Steps:")
position += 1

line = ""
full_str = []
line_len = 100
x = line_len
if line_len < len(your_recipe):
    first_space = your_recipe[(line_len-10):(line_len+10)].index(" ") + line_len - 10
    line = your_recipe[0:first_space]
    full_str.append(line)
    x = first_space
else:
    line = your_recipe

while x < len(your_recipe):
    y = x+line_len
    if y < len(your_recipe):
        index_nearest_space = your_recipe[(y-10):(y+10)].index(" ") + y - 10
    else:
        index_nearest_space = len(your_recipe)
    line = your_recipe[x:index_nearest_space]
    x = index_nearest_space
    full_str.append(line)

print(full_str)

for line in full_str:
    date_canvas.drawString(100, line_y_position(position), f"{line}")
    position += 1

#Meal Section

#QR Section

#Run
date_canvas.showPage()
date_canvas.save()

['Pour all ingredients into shaker with ice cubes. Shake well. Strain into chilled cocktail glass.', ' Garnish with a lemon twist and a maraschino cherry. Serve without a straw.']
